# 单智能体

In [ ]:

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# agents
# pip install -U langgraph   /  pip install -U langchain
# 包括记忆、工具、结构化输出

from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.checkpoint.memory import InMemorySaver

# 记忆
checkpointer = InMemorySaver()

# 结构化输出
class WeatherResponse(BaseModel):
    conditions: str

# 工具
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=chatLLM,
    tools=[get_weather],
    checkpointer=checkpointer,
    response_format=WeatherResponse,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

# Run the agent
# 传入同样的config，就能获得该config下的全部对话
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

# 获取结构化回复
ny_response["structured_response"]

# 多智能体

## 模型

In [ ]:

# 定义状态机
# 状态机通过状态的传递把所有节点chuan'lian

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


In [ ]:

# 定义模型节点

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)

def chatbot(state: State):
    return {"messages": [chatLLM.invoke(state["messages"])]}
    

In [ ]:

# 状态机增加节点
# 参数是：节点名、节点函数

graph_builder.add_node("chatbot", chatbot)


In [ ]:

# 定义开始节点

graph_builder.add_edge(START, "chatbot")


In [ ]:

# 定义结束节点

graph_builder.add_edge("chatbot", END)


In [ ]:

# 编译图

graph = graph_builder.compile()


In [ ]:

# 可视化图

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass
    

In [ ]:

# 运行状态机

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break



## 模型 + 工具·自定义工具类

In [ ]:

# 搜索工具
# pip install -U langchain-tavily

import os

os.environ["TAVILY_API_KEY"] = "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"

from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tools = [tool]
tool.invoke("现在背景时间是那一年几月几日星期几？")


In [ ]:

# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 定义状态机、定义带搜索工具的聊天机器人节点
# llm绑定工具，只能让llm知道这个工具，产生对这个工具的调用信息

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)


In [ ]:

# 定义工具节点
# 定义的这个类是根据llm产生的工具调用信息，调用工具，产生ToolMessage

import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)


In [ ]:

# 定义边、渲染图

def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()


In [ ]:

# 可视化图

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass
    

In [ ]:

# 交互

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break
        

## 模型 + 工具·官方工具类导入

In [ ]:

# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 定义工具·langchain的工具定义规范

from pydantic import BaseModel, Field  # Import Pydantic for data validation
from langchain.tools import tool  # Import the tool decorator from LangChain
from typing import Literal  # Import Literal for type hinting with specific values

class PhoneNumberLookupInput(BaseModel):
    """Input for phone number lookup queries."""
    name: str = Field(description="Name of the person or business to look up")
    location: str = Field(
        default="",
        description="City, state, or country to narrow down the search"
    )
    type: Literal["personal", "business", "government"] = Field(
        default="personal",
        description="Type of phone number to search for"
    )
    include_address: bool = Field(
        default=False,
        description="Whether to include address information in results"
    )


@tool("phone_lookup", description="Looks up phone numbers for people or businesses. Use this when you need contact information.", args_schema=PhoneNumberLookupInput)
def lookup_phone_number(name: str, location: str = "", type: str = "personal", include_address: bool = False) -> str:
    """Find phone numbers based on name and optional filters."""
    # This would contain actual implementation to search a database
    # For demonstration purposes, returning a placeholder
    return f"Found phone number for {name} in {location if location else 'all locations'}: 555-123-4567"
    

In [ ]:

# 建立图

from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 使用Tavily，如果没有显式的注入api key，它会自动从系统环境中读取
# tool = TavilySearch(max_results=2)
tools = [lookup_phone_number]
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[lookup_phone_number])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()


In [ ]:

# 交互

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break
     

## 模型 + 工具 + 记忆

In [ ]:

# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 定义工具·langchain的工具定义规范

from pydantic import BaseModel, Field  # Import Pydantic for data validation
from langchain.tools import tool  # Import the tool decorator from LangChain
from typing import Literal  # Import Literal for type hinting with specific values

class PhoneNumberLookupInput(BaseModel):
    """Input for phone number lookup queries."""
    name: str = Field(description="Name of the person or business to look up")
    location: str = Field(
        default="",
        description="City, state, or country to narrow down the search"
    )
    type: Literal["personal", "business", "government"] = Field(
        default="personal",
        description="Type of phone number to search for"
    )
    include_address: bool = Field(
        default=False,
        description="Whether to include address information in results"
    )


@tool("phone_lookup", description="Looks up phone numbers for people or businesses. Use this when you need contact information.", args_schema=PhoneNumberLookupInput)
def lookup_phone_number(name: str, location: str = "", type: str = "personal", include_address: bool = False) -> str:
    """Find phone numbers based on name and optional filters."""
    # This would contain actual implementation to search a database
    # For demonstration purposes, returning a placeholder
    return f"Found phone number for {name} in {location if location else 'all locations'}: 555-123-4567"
 

In [ ]:

# 建立图

from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 使用Tavily，如果没有显式的注入api key，它会自动从系统环境中读取
# tool = TavilySearch(max_results=2)
tools = [lookup_phone_number]
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[lookup_phone_number])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


In [ ]:

# 创建内存记忆检查点

from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

graph = graph_builder.compile(checkpointer=memory)


In [ ]:

# 创建对话线程

config = {"configurable": {"thread_id": "1"}}


In [ ]:

# 首次交互

user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()


In [ ]:

# 再次交互，测试记忆

user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()
    

In [ ]:

snapshot = graph.get_state(config)
snapshot


In [ ]:

# 获取下一个执行的节点

snapshot.next 


## 人机交互控制

In [ ]:


# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 建立图（包含人工打断工具）

from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


In [ ]:

# 建立记忆检查点

memory = InMemorySaver()

graph = graph_builder.compile(checkpointer=memory)


In [ ]:

# 可视化图

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass
    

In [ ]:

# 交互

user_input = "我需要一些关于构建人工智能代理的专家指导。你能帮我请求协助吗？"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


In [ ]:

# 获取下一个将被执行的节点

snapshot = graph.get_state(config)
snapshot.next


In [ ]:

# 打断后，输入人类反馈，让human_assistance函数执行完

human_response = (
    "我们，这些专家在这里为您提供帮助！我们建议您查看 LangGraph 来构建您的智能代理。"
    "它比简单的自主代理更加可靠且可扩展。"
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()
        

## 更改状态

In [ ]:

# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 定义状态

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str
    

In [ ]:

# 定义工具
# 这个人工干预里面，先打断流程，然后获取人类回复，再然后根据人类回复判断是否做出改变对state里面的三个值重新进行定义，最后更新state

from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)
    

In [ ]:

# 定义图

from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)


In [ ]:

# 交互

user_input = (
    "你能查一下LangGraph是什么时候发布的吗？"
    "当你有答案时，使用 human_assistance 工具进行审核。"
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


In [ ]:

# 打断后，输入人类反馈，让human_assistance函数执行完

human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()
        

In [ ]:

# 查询更改情况

snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}


In [ ]:

# 手动更新状态
# 可以以这种方式更新state中的任何一个键的值

graph.update_state(config, {"name": "LangGraph (library)"})


In [ ]:

# 查询更改情况

snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}


## 时间旅行（重新生成回答）

In [ ]:

# 定义模型

# models
# pip install langchain_openai

from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)


In [ ]:

# 定义图

from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = chatLLM.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)


In [ ]:

# 交互1

config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "我正在学习 LangGraph。"
                    "你能帮我对此做一些调查吗？"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


In [ ]:

# 交互2

events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "是的，那很有帮助。也许我会用它构建一个自主代理！"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()
        

In [ ]:

# 打印全部流程节点

to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state


In [ ]:

# 检查点会在图的每一步保存，这样可以通过调用提取任何一步

print(to_replay.next)
print(to_replay.config)


In [ ]:


# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
# to_replay.config会提供'checkpoint_id': '1f0aefae-4f7d-65e4-8006-41f74551bf4b'，大模型就知道从这个检查点开始添加状态

for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()
